In [1]:
import pandas as pd 

In [3]:
all_data = pd.read_parquet('../data/combined/all_data.parquet')
all_data.head()

,permco,permno,cusip,trading_day_et,prc,ret,retx,vol,shrout,cfacpr,...,f_rp_us_event_count_agg_7d,f_rp_us_event_count_agg_30d,f_rp_us_ess_agg_7d,f_rp_us_ess_agg_30d,f_rp_row_ess,f_rp_row_event_count,f_rp_row_event_count_agg_7d,f_rp_row_event_count_agg_30d,f_rp_row_ess_agg_7d,f_rp_row_ess_agg_30d
0,56028,16852,20009T50,2010-01-04,6.37,-0.004688,-0.004688,18693.0,7572.0,0.006667,...,867.0,3678.0,0.1198,0.0931,0.017,516,1822.0,9560.0,-0.0159,0.0125
1,56249,17279,54414710,2010-01-04,81.52,0.016079,0.016079,1015200.0,160640.0,3.000000,...,867.0,3678.0,0.1198,0.0931,0.017,516,1822.0,9560.0,-0.0159,0.0125
2,16109,19674,45103T10,2010-01-04,21.61,-0.005522,-0.005522,227126.0,58456.0,1.000000,...,867.0,3678.0,0.1198,0.0931,0.017,516,1822.0,9560.0,-0.0159,0.0125
3,37957,19675,83408W10,2010-01-04,59.23,0.034043,0.034043,828329.0,37767.0,1.000000,...,867.0,3678.0,0.1198,0.0931,0.017,516,1822.0,9560.0,-0.0159,0.0125
4,5363,68866,80386630,2010-01-04,41.75,0.045318,0.045318,252800.0,42250.0,1.000000,...,867.0,3678.0,0.1198,0.0931,0.017,516,1822.0,9560.0,-0.0159,0.0125


In [4]:
price_target_summary = pd.read_parquet('../data/analysts_estimate/price_target_summary.parquet')
price_target_summary.head()

,ticker,cusip,meanptg,medptg,stdev,numest,ptghigh,ptglow,statpers,numdown1m,numup1m,numdown4w,numup4w,namedt,nameendt,permco,permno
0,GFGC,36720410,12.0,12.0,NaN,1.0,12.0,12.0,2011-01-20,0.0,0.0,0.0,0.0,2010-07-09,2017-08-03,7953,10001
1,GFGC,36720410,12.0,12.0,NaN,1.0,12.0,12.0,2011-02-17,0.0,0.0,0.0,0.0,2010-07-09,2017-08-03,7953,10001
2,GFGC,36720410,12.0,12.0,NaN,1.0,12.0,12.0,2011-03-17,0.0,0.0,0.0,0.0,2010-07-09,2017-08-03,7953,10001
3,GFGC,36720410,12.5,12.5,NaN,1.0,12.5,12.5,2011-04-14,0.0,1.0,0.0,1.0,2010-07-09,2017-08-03,7953,10001
4,GFGC,36720410,12.5,12.5,NaN,1.0,12.5,12.5,2011-05-19,0.0,0.0,0.0,0.0,2010-07-09,2017-08-03,7953,10001


In [8]:
all_data['trading_day_et'] = pd.to_datetime(all_data['trading_day_et'])
price_target_summary['statpers'] = pd.to_datetime(price_target_summary['statpers'])
all_data.sort_values(by='trading_day_et', inplace=True)

In [9]:
price_target_summary.sort_values(by='statpers', inplace=True)

In [14]:
all_data_with_pt = pd.merge_asof(all_data, price_target_summary[['permno', 'permco', 'statpers', 'medptg']], left_on='trading_day_et', right_on='statpers', by=['permno', 'permco'], tolerance=pd.Timedelta("90d"), direction='backward')

In [16]:
all_data_with_pt.dropna(subset=['medptg'])# .to_parquet('../data/combined/all_data_with_pt_summary.parquet')

,permco,permno,cusip,trading_day_et,prc,ret,retx,vol,shrout,cfacpr,...,f_rp_us_ess_agg_7d,f_rp_us_ess_agg_30d,f_rp_row_ess,f_rp_row_event_count,f_rp_row_event_count_agg_7d,f_rp_row_event_count_agg_30d,f_rp_row_ess_agg_7d,f_rp_row_ess_agg_30d,statpers,medptg
1,51547,91659,01748X10,2010-01-04,48.01000,0.017808,0.017808,268790.0,19910.0,1.0,...,0.1198,0.0931,0.0170,516,1822.0,9560.0,-0.0159,0.0125,2009-12-17,55.00
3,51550,91662,25859810,2010-01-04,10.04000,0.005005,0.005005,167472.0,22092.0,1.0,...,0.1198,0.0931,0.0170,516,1822.0,9560.0,-0.0159,0.0125,2009-12-17,12.00
5,51553,91666,40169210,2010-01-04,5.71000,0.089695,0.089695,29319.0,22975.0,1.0,...,0.1198,0.0931,0.0170,516,1822.0,9560.0,-0.0159,0.0125,2009-12-17,4.25
7,51546,91658,02208R10,2010-01-04,12.69000,0.027530,0.027530,66865.0,26058.0,1.0,...,0.1198,0.0931,0.0170,516,1822.0,9560.0,-0.0159,0.0125,2009-12-17,13.00
8,51555,91668,44980X10,2010-01-04,17.37000,0.038255,0.038255,103796.0,46076.0,1.0,...,0.1198,0.0931,0.0170,516,1822.0,9560.0,-0.0159,0.0125,2009-12-17,18.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21456595,20584,49429,25406710,2024-12-31,431.73999,-0.018361,-0.018929,89094.0,11918.0,1.0,...,-0.0885,0.0115,0.0118,805,4199.0,25774.0,-0.0406,-0.0006,2024-12-19,349.00
21456596,20265,49656,06405810,2024-12-31,76.83000,-0.003631,-0.003631,2380728.0,727078.0,1.0,...,-0.0885,0.0115,0.0118,805,4199.0,25774.0,-0.0406,-0.0006,2024-12-19,84.00
21456597,20557,49680,23585110,2024-12-31,229.55000,-0.000435,-0.000435,2077923.0,722275.0,1.0,...,-0.0885,0.0115,0.0118,805,4199.0,25774.0,-0.0406,-0.0006,2024-12-19,285.00
21456599,20238,44601,05361110,2024-12-31,187.13000,-0.000587,-0.000587,452740.0,80346.0,1.0,...,-0.0885,0.0115,0.0118,805,4199.0,25774.0,-0.0406,-0.0006,2024-12-19,236.50


In [22]:
price_target_detail = pd.read_parquet('../data/analysts_estimate/price_target_detail.parquet')
# if ann is after 4pm, then count it to next day
price_target_detail.head()

,ticker,estimid,alysnam,value,cusip,amaskcd,ann,act,ann_deemed_date,namedt,nameendt,permco,permno
0,MSO,BEAR,GRUNEICH,33.0,57308310,1797.0,2000-01-03,2000-01-04 13:46:06,2000-01-03,1999-10-19,2002-01-01,36347,87277
1,KRN,BEAR,GRUNEICH,68.0,49904010,1797.0,2000-01-03,2000-01-03 00:00:00,2000-01-03,1986-05-01,2002-01-01,21061,48960
2,TGNT,LAWRENCE,FIFER B,70.0,87959Y10,53300.0,2000-01-03,2000-01-03 00:00:00,2000-01-03,1997-11-21,2001-05-10,15845,85581
3,MXIM,LAWRENCE,KLAUER E,52.0,57772K10,48357.0,2000-01-03,2000-01-03 00:00:00,2000-01-03,1988-02-29,2004-06-09,9698,11896
4,SKYW,LAWRENCE,DONOFRIO S,35.0,83087910,25414.0,2000-01-03,2000-01-03 00:00:00,2000-01-03,1999-07-01,2002-11-21,8340,10421


In [24]:
price_target_detail['ann_deemed_date'] = pd.to_datetime(price_target_detail['ann_deemed_date'])
price_target_detail_all_data = pd.merge_asof(price_target_detail[['permno', 'permco', 'ann_deemed_date', 'value']], all_data, left_on='ann_deemed_date', right_on='trading_day_et', by=['permno', 'permco'], tolerance=pd.Timedelta("10d"), direction='forward')

In [ ]:
price_target_detail_all_data.dropna(subset=['prc']).to_parquet('../data/combined/price_target_detail_all_data.parquet')

,permno,permco,ann_deemed_date,value,cusip,trading_day_et,prc,ret,retx,vol,...,f_rp_us_event_count_agg_7d,f_rp_us_event_count_agg_30d,f_rp_us_ess_agg_7d,f_rp_us_ess_agg_30d,f_rp_row_ess,f_rp_row_event_count,f_rp_row_event_count_agg_7d,f_rp_row_event_count_agg_30d,f_rp_row_ess_agg_7d,f_rp_row_ess_agg_30d
457720,52038,21698,2009-12-25,26.0,87182910,2010-01-04,28.10,0.005727,0.005727,2459600.0,...,867.0,3678.0,0.1198,0.0931,0.0170,516,1822.0,9560.0,-0.0159,0.0125
457721,14702,194,2009-12-25,16.0,03822210,2010-01-04,14.30,0.025825,0.025825,18713906.0,...,867.0,3678.0,0.1198,0.0931,0.0170,516,1822.0,9560.0,-0.0159,0.0125
457722,90209,45283,2009-12-27,3.0,46820210,2010-01-04,4.43,0.006818,0.006818,1822200.0,...,867.0,3678.0,0.1198,0.0931,0.0170,516,1822.0,9560.0,-0.0159,0.0125
457723,61399,2709,2009-12-28,25.0,54866110,2010-01-04,23.16,-0.009833,-0.009833,9616800.0,...,867.0,3678.0,0.1198,0.0931,0.0170,516,1822.0,9560.0,-0.0159,0.0125
457724,66181,5085,2009-12-28,32.0,43707610,2010-01-04,28.67,-0.008987,-0.008987,13121400.0,...,867.0,3678.0,0.1198,0.0931,0.0170,516,1822.0,9560.0,-0.0159,0.0125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784859,93098,53266,2024-12-31,176.0,44857910,2024-12-31,156.98,-0.007461,-0.007461,264162.0,...,1765.0,13857.0,-0.0885,0.0115,0.0118,805,4199.0,25774.0,-0.0406,-0.0006
1784860,16499,55807,2024-12-31,22.0,70051710,2024-12-31,14.07,0.000680,-0.043508,5419212.0,...,1765.0,13857.0,-0.0885,0.0115,0.0118,805,4199.0,25774.0,-0.0406,-0.0006
1784861,85913,30005,2024-12-31,295.0,57190320,2024-12-31,278.94,-0.004887,-0.004887,736598.0,...,1765.0,13857.0,-0.0885,0.0115,0.0118,805,4199.0,25774.0,-0.0406,-0.0006
1784862,90394,45703,2024-12-31,12.0,86789210,2024-12-31,11.84,0.008453,0.000845,1491943.0,...,1765.0,13857.0,-0.0885,0.0115,0.0118,805,4199.0,25774.0,-0.0406,-0.0006
